In [16]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## For Just One Piece

In [17]:
# Select a prefix:

# prefix = 'Music_Files/'
prefix = 'https://crimproject.org/mei/'

# Add your filename here

mei_file = 'CRIM_Model_0050.mei'

url = prefix + mei_file

piece = importScore(url)

print(piece.metadata)

Memoized piece detected.
{'title': 'Domine Dominus noster', 'composer': 'Roland de  Lassus'}


## Find ngrams, find HR, and filter the dataframe to only the HR modules

In [22]:
hr = piece.homorhythm()
hr['voice_match'] = hr['active_voices'] == hr['active_syll_voices']
hr_full = hr[hr['voice_match'] == True]
hr_staggered = hr[hr['voice_match'] == False]
hr_list = hr_full.index.get_level_values(2).to_list()
ng = piece.ngrams()
hr_ng = ng.filter(hr_list, axis=0)
out = piece.detailIndex(hr_ng)


In [30]:
out.head()

Bassus_Quinta   Bassus_Tenor     Bassus_Altus     Bassus_Sexta  \
Measure Beat                                                                  
1       1.0   3_-4, 5_1, 5   5_-4, 8_1, 8   8_-4, 10_1, 10   8_-4, 12_1, 12   
        2.0   5_1, 5_-5, 8  8_1, 8_-5, 12  10_1, 10_-5, 15  12_1, 12_-5, 19   
        2.5   5_-5, 8_4, 5  8_-5, 12_4, 8  10_-5, 15_4, 10  12_-5, 19_4, 12   
        3.0    8_4, 5_1, 5   12_4, 8_1, 8   15_4, 10_1, 10   19_4, 12_1, 12   
        4.0   5_1, 5_-5, 8  8_1, 8_-5, 12  10_1, 10_-5, 15  12_1, 12_-5, 17   

                Bassus_Cantus  Quinta_Tenor  Quinta_Altus   Quinta_Sexta  \
Measure Beat                                                               
1       1.0   12_-4, 15_1, 15  3_-2, 4_1, 4  6_-2, 6_1, 6   6_-2, 8_1, 8   
        2.0   15_1, 15_-5, 17  4_1, 4_-2, 5  6_1, 6_-2, 8  8_1, 8_-2, 12   
        2.5   15_-5, 17_4, 15  4_-2, 5_1, 4  6_-2, 8_1, 6  8_-2, 12_1, 8   
        3.0    17_4, 15_1, 15   5_1, 4_1, 4   8_1, 6_1, 6   12_1, 8_1, 8   
        4.0   15_1, 15_-5, 19  4_1, 4_-2, 5  6_1, 6_-2, 8  8_1, 8_-2, 10   

                Quinta_Cantus   Tenor_Altus   Tenor_Sexta  Tenor_Cantus  \
Measure Beat                                                              
1       1.0   10_-2, 11_1, 11   4_1, 3_1, 3   4_1, 5_1, 5   8_1, 8_1, 8   
        2.0   11_1, 11_-2, 10   3_1, 3_1, 4   5_1, 5_1, 8   8_1, 8_1, 6   
        2.5   11_-2, 10_1, 11  3_1, 4_-2, 3  5_1, 8_-2, 5  8_1, 6_-2, 8   
        3.0    10_1, 11_1, 11  4_-2, 3_1, 3  8_-2, 5_1, 5  6_-2, 8_1, 8   
        4.0   11_1, 11_-2, 12   3_1, 3_1, 4   5_1, 5_1, 6   8_1, 8_1, 8   

               Altus_Sexta  Altus_Cantus   Sexta_Cantus  
Measure Beat                                             
1       1.0   1_-2, 3_1, 3  5_-2, 6_1, 6    5_2, 4_1, 4  
        2.0    3_1, 3_2, 5   6_1, 6_2, 3   4_1, 4_4, -3  
        2.5   3_2, 5_-3, 3  6_2, 3_-3, 6  4_4, -3_-5, 4  
        3.0   5_-3, 3_1, 3  3_-3, 6_1, 6  -3_-5, 4_1, 4  
        4.0    3_1, 3_2, 3   6_1, 6_2, 5    4_1, 4_2, 3

## The Same with a Corpus.

* It makes more sense to produce a series of individual files
* with the filtered ngrams.  Why?  Because otherwise
* You will one DF with EVERY combination of voices, which will be rather strange.
* This script takes a list of pieces and makes ONE filtered CSV for each piece


In [26]:
piece_list = ['CRIM_Mass_0049_1',
             'CRIM_Mass_0049_2',
             'CRIM_Model_0050']

In [27]:
prefix = 'https://crimproject.org/mei/'
suffix = '.mei'

for item in piece_list:
    url = prefix + item + suffix
    piece = importScore(url)
    hr = piece.homorhythm()
    hr['voice_match'] = hr['active_voices'] == hr['active_syll_voices']
    hr_full = hr[hr['voice_match'] == True]
    hr_staggered = hr[hr['voice_match'] == False]
    hr_list = hr_full.index.get_level_values(2).to_list()
    ng = piece.ngrams()
    hr_ng = ng.filter(hr_list, axis=0)
    out = piece.detailIndex(hr_ng)
    out.to_csv('saved_csv/' + item + '.csv')
    

Successfully imported https://crimproject.org/mei/CRIM_Mass_0049_1.mei
Successfully imported https://crimproject.org/mei/CRIM_Mass_0049_2.mei
Memoized piece detected.


### Review CSV and Search for Ngrams

* Now review the CSV files for the locations that interest you
* Take note of the Modules, and use them as 'my_string' in the following cell
* Of course you will need to select just the piece you want

In [28]:
# prefix = 'Music_Files/'
prefix = 'https://crimproject.org/mei/'

# Add your filename here

mei_file = 'CRIM_Model_0050.mei'

url = prefix + mei_file

piece = importScore(url)



my_string = '8_1, 8_1, 8'

ngrams = piece.ngrams()

filtered_ngrams = ngrams[ngrams.apply(lambda x: x.astype(str).str.contains(my_string).any(), axis=1)]#.copy()
beats_measures_mod = piece.detailIndex(filtered_ngrams, offset=True)
beats_measures_mod.fillna("-").applymap(str).style.applymap(lambda x: "background: #ccebc5" if re.search(my_string, x) else "")


Memoized piece detected.
